In [1]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
import math
from functools import partial
from open3d.t.geometry import TriangleMesh

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
dataname = "/home/chris/Code/PointClouds/data/ply/CircularVentilationGrateExtraCleanedFull.ply"
pcd = o3d.io.read_point_cloud(dataname)

In [3]:
pcd_center = pcd.get_center()
pcd.translate(-pcd_center)

PointCloud with 2116800 points.

In [4]:
#Outlier removal

nn = 16
std_multiplier = 10

filtered_pcd = pcd.remove_statistical_outlier(nn,std_multiplier)
outliers = pcd.select_by_index(filtered_pcd[1], invert = True)
outliers.paint_uniform_color([1,0,0])
filtered_pcd = filtered_pcd[0]

o3d.visualization.draw_geometries([filtered_pcd, outliers])

In [5]:
#Downsampling

voxel_size = 0.01
pcd_downsampled = filtered_pcd.voxel_down_sample(voxel_size=voxel_size)
print(f'number of points: {len(pcd_downsampled.points)}')
#o3d.visualization.draw_geometries([pcd_downsampled])

number of points: 2114266


In [6]:
#Extract normals

nn_distance = np.mean([pcd.compute_nearest_neighbor_distance()])
radius_normals = nn_distance*4

pcd_downsampled.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normals, max_nn=16), fast_normal_computation=True)

pcd_downsampled.paint_uniform_color([0.6,0.6,0.6])
#o3d.visualization.draw_geometries([pcd_downsampled, outliers])

PointCloud with 2114266 points.

In [7]:
# Extracting and Setting Parameters

front =  [-0.47452876114542436, 0.57451207113849134, -0.66690204300328082]
lookat = [-6.3976792217838847, 20.927374714553928, 18.659758576873813]
up =  [-0.056918726368614558, -0.77607794684805009, -0.62806311705487861]
zoom = 0.69999999999999996

pcd = pcd_downsampled

#o3d.visualization.draw_geometries([pcd_downsampled], zoom=zoom, front=front, lookat=lookat, up=up)

In [8]:
#Initial test Ransac to find optimal pt_to_plane_dist
pt_to_plane_dist = 0.5

plane_model, inliers = pcd.segment_plane(distance_threshold=pt_to_plane_dist,ransac_n=3,num_iterations=1000)
[a,b,c,d] = plane_model
print(f'Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0')

inlier_cloud = pcd.select_by_index(inliers)
outlier_cloud = pcd.select_by_index(inliers, invert=True)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud.paint_uniform_color([0.6, 0.6, 0.6])

o3d.visualization.draw_geometries([inlier_cloud,outlier_cloud], zoom=zoom, front=front, lookat=lookat, up=up)

Plane equation: 0.00x + 0.00y + 1.00z + -0.71 = 0


In [9]:
# Multi-order Ransac

max_plane_idx = 7
pt_to_plane_dist = 0.4

segment_models = {}
segments = {}
main_surface_idx = 0
largest_surface_points = 0
rest = pcd

for i in range(max_plane_idx):
    print(f'Run {i}/{max_plane_idx} started. ', end='')
    colors = plt.get_cmap("tab20")(i)
    segment_models[i], inliers = rest.segment_plane(distance_threshold=pt_to_plane_dist,ransac_n=3,num_iterations=50000)
    segments[i] = rest.select_by_index(inliers)
    if len(segments[i].points) > largest_surface_points:
        largest_surface_points = len(segments[i].points) 
        main_surface_idx = i
    segments[i].paint_uniform_color(list(colors[:3]))
    rest = rest.select_by_index(inliers, invert=True)
    print('Done')

print('Largest surface found with segment idx', main_surface_idx)
print(segment_models)
o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)]+[rest],zoom=zoom,front=front,lookat=lookat,up=up)

Run 0/7 started. Done
Run 1/7 started. Done
Run 2/7 started. Done
Run 3/7 started. Done
Run 4/7 started. Done
Run 5/7 started. Done
Run 6/7 started. Done
Largest surface found with segment idx 0
{0: array([ 0.00142063,  0.00389943,  0.99999139, -0.71307748]), 1: array([ 0.50687498, -0.26852217,  0.81912978, -0.35052285]), 2: array([ 0.00965695, -0.57646156,  0.8170672 , -0.58675831]), 3: array([0.49419264, 0.29888231, 0.8163596 , 0.14452459]), 4: array([-0.50377458,  0.27481611,  0.81895499, -2.05130758]), 5: array([-0.01068348,  0.57623558,  0.81721382, -0.6913764 ]), 6: array([-0.49025187, -0.28881261,  0.82233836, -1.46767155])}


In [10]:
#Calculate angles between main surface and extrusions

angles_rad = {}
angles_deg = {}

for i in range(len(segment_models)):
    if i != main_surface_idx:
        dot_product = np.dot(segment_models[main_surface_idx][:3], segment_models[i][:3])
        angles_rad[i] = np.arccos(np.clip(dot_product, -1, 1))
        angles_deg[i] = angles_rad[i] * 180 / np.pi
        #print(angles_deg[i])

print(angles_deg)


{1: 35.035577277492415, 2: 35.42983788842751, 3: 35.092985560953785, 4: 34.98482961513364, 5: 34.97136599490421, 6: 34.864260696490916}


In [11]:
#Find plane-to-plane intersection lines (the lines defined by the 
#cross_product for directionality and the common_point for line placement)

intersection_lines = {}

for i in range(len(segment_models)):
    if i != main_surface_idx:
        cross_product = np.cross(segment_models[main_surface_idx][:3], segment_models[i][:3])
        if abs(abs(cross_product[2]) < 0.0001):
            print(cross_product)
            common_point = np.linalg.solve([segment_models[main_surface_idx][1:3], segment_models[i][1:3]],
                                    [-segment_models[main_surface_idx][-1], -segment_models[i][-1]])
        else:
            common_point = np.linalg.solve([segment_models[main_surface_idx][:2], segment_models[i][:2]],
                                    [-segment_models[main_surface_idx][-1], -segment_models[i][-1]])

        common_point = np.asarray([common_point[0], common_point[1], 0])
            
        intersection_lines[i] = [cross_product,common_point]

print(intersection_lines)


{1: [array([ 0.27171399,  0.50570693, -0.00235799]), array([ 81.78303063, 153.07208937,   0.        ])], 2: [array([ 0.57964269,  0.00849612, -0.0008566 ]), array([482.54815367,   7.06584395,   0.        ])], 3: [array([-0.2956964 ,  0.49302864, -0.00150247]), array([-142.22603276,  234.68278952,    0.        ])], 4: [array([-0.27162029, -0.50493368,  0.00235485]), array([ 79.82102833, 153.78688358,   0.        ])], 5: [array([-0.57304395, -0.01184435,  0.00086028]), array([474.50217466,   9.9971449 ,   0.        ])], 6: [array([ 0.29201678, -0.49141589,  0.0015014 ]), array([-140.98053154,  234.22903008,    0.        ])]}


In [12]:
# Find the anchor points of the intersections of the planes and visualize

anchor_points = {}
for i in range(len(segment_models)):
    if i != main_surface_idx:
        segment_center = segments[i].get_center()
        b_vec = [sc - il for sc, il in zip(segment_center, intersection_lines[i][1])]
        a_vec = intersection_lines[i][0]/np.linalg.norm(intersection_lines[i][0])
        p_vec = np.dot(a_vec,b_vec)/np.dot(a_vec,a_vec) * a_vec
        anchor_points[i] = intersection_lines[i][1] + p_vec

#print(anchor_points)

sphere_radius = 5  # Adjust the radius based on your scale
spheres = []

for point in anchor_points.values():
    sphere = o3d.geometry.TriangleMesh.create_sphere(radius=sphere_radius)
    sphere.translate(point)  # Move the sphere to the anchor point
    sphere.paint_uniform_color([1, 0, 0])  # Red color for visibility
    spheres.append(sphere)

# Visualize
o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)]+[rest]+spheres, zoom=zoom, front=front, lookat=lookat, up=up)

In [13]:
# Perform edge detection around anchor points

pcd_tree = o3d.geometry.KDTreeFlann(pcd)
eig_val_dict = {}
plane_dir_dict = {}
sample_dist = 0.7
number_of_neighbors = 10
sampled_neighborhoods = {}
initial_eigen_agg = 0
forward = True
eigen_threshold = 100000
angle_threshold = 0.11
radius = 2

for i in range(len(segment_models)):
    if i != main_surface_idx:
        print(f'Segment {i}')
        print(50*'-')
        it = 0
        forward = True
        while True:
            if forward:
                sampled_anchor_point = anchor_points[i] + (it*sample_dist)*intersection_lines[i][0]/np.linalg.norm(intersection_lines[i][0])
            else:
                sampled_anchor_point = anchor_points[i] - (it*sample_dist)*intersection_lines[i][0]/np.linalg.norm(intersection_lines[i][0])
            [k, idx, _] = pcd_tree.search_radius_vector_3d(sampled_anchor_point, radius)
            sampled_neighborhoods[it] = idx
            if k < 8:
                if forward:
                    print('Boundary reached. Changing direction to backwards')
                    it = 0
                    forward = False
                    eig_val_dict.clear()
                    continue
                else:
                    print('Boundary reached. Moving on to next segment if there is one')
                    break

            neighbor_coordinates = np.asarray(pcd.points)[idx[1:], :]

            mu = np.mean(neighbor_coordinates, axis=0)
            norm = neighbor_coordinates - mu
            cov = np.cov(norm.T)
            eig_val, eig_vec = np.linalg.eig(cov)
            sorted_idx = np.argsort(eig_val)[::-1]
            eig_val = eig_val[sorted_idx]
            eig_vec = eig_vec[:, sorted_idx]

            eig_val_norm = eig_val.copy()
            for z in range(len(eig_val)):
                eig_val_norm[z] = np.exp(eig_val[z])/np.sum(np.exp(eig_val))
            eig_val_dict[it] = eig_val_norm

            plane_dir = np.cross(eig_vec[:, 0], eig_vec[:, 1])
            if it > 0:
                previous_plane_dir = plane_dir_dict[it-1]
                if np.dot(plane_dir,previous_plane_dir)<0:
                    plane_dir = -plane_dir
            plane_dir_dict[it] = plane_dir

            eigen_agg = 0
            angle_agg = 0

            if it > number_of_neighbors:
                for eigen_index in range(len(eig_val_norm)):
                    for prev_iteration in range(number_of_neighbors):
                        diff = eig_val_norm[eigen_index] - eig_val_dict[it-(prev_iteration+1)][eigen_index]
                        eigen_agg += (diff/sample_dist)**2

            
                for prev_iteration in range(number_of_neighbors):
                    #Calculate aggragate for the planes of the ellipsoids
                    dot_product = np.dot(plane_dir, plane_dir_dict[it-(prev_iteration+1)])
                    norm_1 = np.linalg.norm(plane_dir)
                    norm_2 = np.linalg.norm(plane_dir_dict[it-(prev_iteration+1)])
                    plane_angle = np.arccos(np.clip(dot_product/(norm_1*norm_2), -1, 1))
                    angle_agg += plane_angle
                angle_agg /= number_of_neighbors
            
                if it == number_of_neighbors + 1:
                    initial_eigen_agg = eigen_agg
                    initial_angle_agg = angle_agg
                if eigen_agg > eigen_threshold or angle_agg > angle_threshold:
                    if forward:
                        print(f'broke edge moving forwards. Initial and current eigen agg: [{initial_eigen_agg},{eigen_agg}]. Initial and current angle agg: [{initial_angle_agg}, {angle_agg}]')
                        print('')
                        it = 0
                        forward = False
                        eig_val_dict.clear()
                        continue
                    else:
                        print(f'broke edge moving forwards. Initial and current eigen agg: [{initial_eigen_agg},{eigen_agg}]. Initial and current angle agg: [{initial_angle_agg}, {angle_agg}]')
                        print('')
                        break
                else: 
                    #Paint Neighborhood
                    #np.asarray(pcd.colors)[sampled_neighborhoods[it][1:], :] = (0, 1, 0)
                    #Paint only points
                    nearest_point_idx = idx[0]  # The index of the sampled anchor point
                    np.asarray(pcd.colors)[nearest_point_idx, :] = (0, 1, 0)
            else: 
                #Paint Neighborhood
                #np.asarray(pcd.colors)[sampled_neighborhoods[it][1:], :] = (0, 1, 0)
                #Paint only points
                nearest_point_idx = idx[0]  # The index of the sampled anchor point
                np.asarray(pcd.colors)[nearest_point_idx, :] = (0, 1, 0)
            it += 1
        eig_val_dict.clear()
        


o3d.visualization.draw_geometries([pcd], zoom=zoom, front=front, lookat=lookat, up=up)

Segment 1
--------------------------------------------------
broke edge moving forwards. Initial and current eigen agg: [0.0004690298637203134,0.00048353738475494]. Initial and current angle agg: [0.0072627979395240546, 0.11328961294307087]

broke edge moving forwards. Initial and current eigen agg: [0.0015073882267641102,0.0017810122597436543]. Initial and current angle agg: [0.010219946579746033, 0.1119329201376007]

Segment 2
--------------------------------------------------
Boundary reached. Changing direction to backwards
broke edge moving forwards. Initial and current eigen agg: [0.0009948147358655461,0.0010077663050479925]. Initial and current angle agg: [0.010858782885064668, 0.1438352457909385]

Segment 3
--------------------------------------------------
broke edge moving forwards. Initial and current eigen agg: [0.00019836269654843516,0.0018704698089314592]. Initial and current angle agg: [0.004212441932037527, 0.15680375725726]

broke edge moving forwards. Initial and curr

In [14]:
# # Parameter testing
# sample_dist_range = [0.15, 0.16, 0.17, 0.2, 0.22]  # Example values for sample_dist
# number_of_neighbors_range = [15, 17, 18, 20]  # Example values for number_of_neighbors
# threshold_range = [15, 17, 20, 22, 24, 25]  # Example values for threshold
# radius_range = [6, 7]  # Example values for radius
# pcd_tree = o3d.geometry.KDTreeFlann(pcd)
# # Iterate over parameter combinations
# for sample_dist in sample_dist_range:
#     for number_of_neighbors in number_of_neighbors_range:
#         for threshold in threshold_range:
#             for radius in radius_range:
#                 np.asarray(pcd.colors)[:, :] = [0.6, 0.6, 0.6]  # Set all colors to white
#                 print(f'Sample Dist: {sample_dist}, Number of Neighbors: {number_of_neighbors}, Threshold: {threshold}, Radius: {radius}')
                
#                 # Perform edge detection around anchor points
#                 eig_val_dict = {}
#                 sampled_neighborhoods = {}
#                 forward = True
#                 initial_agg = 0
                
#                 for i in range(len(segment_models)):
#                     if i != main_surface_idx:
#                         #print(f'Segment {i}')
#                         #print(50*'-')
#                         it = 0
#                         forward = True
#                         while True:
#                             if forward:
#                                 sampled_anchor_point = anchor_points[i] + (it*sample_dist)*intersection_lines[i][0]/np.linalg.norm(intersection_lines[i][0])
#                             else:
#                                 sampled_anchor_point = anchor_points[i] - (it*sample_dist)*intersection_lines[i][0]/np.linalg.norm(intersection_lines[i][0])
#                             [k, idx, _] = pcd_tree.search_radius_vector_3d(sampled_anchor_point, radius)
#                             sampled_neighborhoods[it] = idx
#                             if k < 10:
#                                 if forward:
#                                     it = 0
#                                     forward = False
#                                     eig_val_dict.clear()
#                                     continue
#                                 else:
#                                     break
#                             neighbor_coordinates = np.asarray(pcd.points)[idx[1:], :]

#                             mu = np.mean(neighbor_coordinates, axis=0)
#                             norm = neighbor_coordinates - mu
#                             cov = np.cov(norm.T)
                            
#                             eig_val, eig_vec = np.linalg.eig(cov.T)
#                             sorted_idx = np.argsort(eig_val)[::-1]
#                             eig_val = eig_val[sorted_idx]
#                             eig_val_norm = eig_val.copy()
#                             for z in range(len(eig_val)):
#                                 eig_val_norm[z] = np.exp(eig_val[z])/np.sum(np.exp(eig_val))
#                             #eig_vec = eig_vec[:, sorted_idx]
#                             eig_val_dict[it] = eig_val_norm

#                             agg = 0
#                             if it > number_of_neighbors:
#                                 for eigen_index in range(len(eig_val_norm)):
#                                     for prev_iteration in range(number_of_neighbors):
#                                         #print(f'it-(prev_iteration+1)= {it-(prev_iteration+1)}')
#                                         diff = eig_val_norm[eigen_index] - eig_val_dict[it-(prev_iteration+1)][eigen_index]
#                                         agg += (diff/sample_dist)**2
#                                 #print(agg)
#                                 if it == number_of_neighbors + 1:
#                                     initial_agg = agg
#                                 if agg > threshold:
#                                     if forward:
#                                         #print(f'broke edge moving forwards with initial agg: {initial_agg} and current agg: {agg}')
#                                         #print('')
#                                         it = 0
#                                         forward = False
#                                         eig_val_dict.clear()
#                                         continue
#                                     else:
#                                         #print(f'broke edge moving backwards with initial agg: {initial_agg} and current agg: {agg}')
#                                         #print('')
#                                         break
#                                 else: 
#                                     np.asarray(pcd.colors)[sampled_neighborhoods[it-number_of_neighbors][1:], :] = (0, 1, 0)
                            
#                             it += 1
#                         eig_val_dict.clear()

#                 # Visualize the result
#                 o3d.visualization.draw_geometries([pcd], zoom=zoom, front=front, lookat=lookat, up=up)



In [15]:
# # Step by step visualization

# # Perform edge detection around anchor points

# pcd_tree = o3d.geometry.KDTreeFlann(pcd)
# eig_val_dict = {}
# sample_dist = 0.15
# number_of_neighbors = 17
# sampled_neighborhoods = {}
# initial_agg = 0
# forward = True
# threshold = 17
# radius = 7

# for i in range(len(segment_models)):
#     if i != main_surface_idx:
#         print(f'Segment {i}')
#         print(50*'-')
#         it = 0
#         forward = True
#         while True:
#             if forward:
#                 sampled_anchor_point = anchor_points[i] + (it*sample_dist)*intersection_lines[i][0]/np.linalg.norm(intersection_lines[i][0])
#             else:
#                 sampled_anchor_point = anchor_points[i] - (it*sample_dist)*intersection_lines[i][0]/np.linalg.norm(intersection_lines[i][0])
#             [k, idx, _] = pcd_tree.search_radius_vector_3d(sampled_anchor_point, radius)
#             sampled_neighborhoods[it] = idx
#             if k < 10:
#                 if forward:
#                     it = 0
#                     forward = False
#                     eig_val_dict.clear()
#                     continue
#                 else:
#                     break
#             neighbor_coordinates = np.asarray(pcd.points)[idx[1:], :]

#             mu = np.mean(neighbor_coordinates, axis=0)
#             norm = neighbor_coordinates - mu
#             cov = np.cov(norm.T)
            
#             eig_val, eig_vec = np.linalg.eig(cov.T)
#             sorted_idx = np.argsort(eig_val)[::-1]
#             eig_val = eig_val[sorted_idx]
#             eig_val_norm = eig_val.copy()
#             for z in range(len(eig_val)):
#                 eig_val_norm[z] = np.exp(eig_val[z])/np.sum(np.exp(eig_val))
#             #eig_vec = eig_vec[:, sorted_idx]
#             eig_val_dict[it] = eig_val_norm

#             agg = 0
#             if it > number_of_neighbors:
#                 for eigen_index in range(len(eig_val_norm)):
#                     for prev_iteration in range(number_of_neighbors):
#                         #print(f'it-(prev_iteration+1)= {it-(prev_iteration+1)}')
#                         diff = eig_val_norm[eigen_index] - eig_val_dict[it-(prev_iteration+1)][eigen_index]
#                         agg += (diff/sample_dist)**2
#                 print(agg)
#                 if it == number_of_neighbors + 1:
#                     initial_agg = agg
#                 if agg > threshold:
#                     if forward:
#                         print(f'broke edge moving forwards with initial agg: {initial_agg} and current agg: {agg}')
#                         print('')
#                         it = 0
#                         forward = False
#                         eig_val_dict.clear()
#                         continue
#                     else:
#                         print(f'broke edge moving backwards with initial agg: {initial_agg} and current agg: {agg}')
#                         print('')
#                         break
#                 else: 
#                     np.asarray(pcd.colors)[sampled_neighborhoods[it-number_of_neighbors][1:], :] = (0, 1, 0)
            
#             it += 1
#         eig_val_dict.clear()
        


# o3d.visualization.draw_geometries([pcd], zoom=zoom, front=front, lookat=lookat, up=up)

In [16]:
# Callback test

pcd_tree = o3d.geometry.KDTreeFlann(pcd)
eig_val_dict = {}
sample_dist = 1
number_of_neighbors = 10
sampled_neighborhoods = {}
initial_eigen_agg = 0
forward = True
eigen_threshold = 100000
angle_threshold = 0.11
radius = 2

class CallbackState:
    def __init__(self, anchor_points, pcd_tree, 
                 eigen_threshold,number_of_neighbors, 
                 sample_dist, radius, intersection_lines, pcd, angle_threshold):

        self.pcd_tree=pcd_tree
        self.counter = 0  # This parameter changes each time the function is called
        self.forward=True
        self.anchor_points=anchor_points
        self.eigen_threshold=eigen_threshold
        self.angle_threshold=angle_threshold
        self.number_of_neighbors=number_of_neighbors
        self.sample_dist=sample_dist
        self.radius=radius
        self.intersection_lines=intersection_lines
        self.segment_models = segment_models
        self.pcd = pcd
        self.sampled_neighborhoods={}
        self.coloring_sampled_neighborhoods={}
        self.initial_eigen_agg=0
        self.initial_angle_agg=0
        self.eig_val_dict={}
        self.segment_painted_points = {}
        self.plane_dir_dict={}
        self.it=0
        self.seg_id = 1
        self.pcd_center = pcd.get_center()
        self.eigen_agg_text_mesh: TriangleMesh = o3d.t.geometry.TriangleMesh.create_text('Eigen agg =', depth=4).to_legacy()
        self.angle_agg_text_mesh: TriangleMesh = o3d.t.geometry.TriangleMesh.create_text('Angle agg =', depth=4).to_legacy()
        self.line_geometry = []

    def calculate_lines(self):
        for seg in self.segment_painted_points:
            total_distance = self.segment_painted_points[seg]*self.sample_dist
            print(f'Total distance for segment {seg} = {total_distance}')
        
    def key_callback(self, vis):
        #print(self.it)
        if self.seg_id not in self.segment_painted_points:
            self.segment_painted_points[self.seg_id] = 0
        if self.seg_id > 6:
            self.calculate_lines()
            #print('No more segments')
            return
        if self.forward:
            sampled_anchor_point = self.anchor_points[self.seg_id] + (self.it*self.sample_dist)*self.intersection_lines[self.seg_id][0]/np.linalg.norm(self.intersection_lines[self.seg_id][0])
        else:
            sampled_anchor_point = self.anchor_points[self.seg_id] - (self.it*self.sample_dist)*self.intersection_lines[self.seg_id][0]/np.linalg.norm(self.intersection_lines[self.seg_id][0])
        [k, idx, _] = self.pcd_tree.search_radius_vector_3d(sampled_anchor_point, self.radius)
        self.sampled_neighborhoods[self.it] = idx
        [k_color, idx_color, _] = self.pcd_tree.search_radius_vector_3d(sampled_anchor_point, 2)
        self.coloring_sampled_neighborhoods[self.it] = idx_color
        if k < 8:
            if self.forward:
                print('Point cloud boundary reached. Turning backwards')
                self.segment_painted_points[self.seg_id] += self.it
                self.it = 0
                self.forward = False
                self.eig_val_dict.clear()
                return
            else:
                self.forward = True
                print('Point cloud boundary reached. Moving on to next segment')
                self.segment_painted_points[self.seg_id] += self.it
                self.seg_id += 1
                self.it = 0
                return

        neighbor_coordinates = np.asarray(self.pcd.points)[idx[1:], :]

        mu = np.mean(neighbor_coordinates, axis=0)
        norm = neighbor_coordinates - mu
        cov = np.cov(norm.T)
        eig_val, eig_vec = np.linalg.eig(cov)
        sorted_idx = np.argsort(eig_val)[::-1]
        eig_val = eig_val[sorted_idx]
        eig_vec = eig_vec[:, sorted_idx]
        #print(eig_val)
        eig_val_norm = eig_val.copy()
        for z in range(len(eig_val)):
            eig_val_norm[z] = np.exp(eig_val[z])/np.sum(np.exp(eig_val))
        self.eig_val_dict[self.it] = eig_val_norm

        plane_dir = np.cross(eig_vec[:, 0], eig_vec[:, 1])
        #print(f'Plane dict: {self.plane_dir_dict}')
        if self.it == 0:
            self.plane_dir_dict[self.it] = plane_dir
        elif self.it > 0:
            previous_plane_dir = self.plane_dir_dict[self.it-1]
            if np.dot(plane_dir,previous_plane_dir)<0:
                plane_dir = -plane_dir
            self.plane_dir_dict[self.it] = plane_dir


        eigen_agg = 0
        angle_agg = 0
        if self.it > self.number_of_neighbors:
            for eigen_index in range(len(eig_val_norm)):
                for prev_iteration in range(self.number_of_neighbors):
                    #Calculate aggragate for the eigenvalues
                    eigen_diff = eig_val_norm[eigen_index] - self.eig_val_dict[self.it-(prev_iteration+1)][eigen_index]
                    eigen_agg += (eigen_diff/self.sample_dist)**2
                    
            for prev_iteration in range(self.number_of_neighbors):
                #Calculate aggragate for the planes of the ellipsoids
                dot_product = np.dot(plane_dir, self.plane_dir_dict[self.it-(prev_iteration+1)])
                norm_1 = np.linalg.norm(plane_dir)
                norm_2 = np.linalg.norm(self.plane_dir_dict[self.it-(prev_iteration+1)])
                plane_angle = np.arccos(np.clip(dot_product/(norm_1*norm_2), -1, 1))
                angle_agg += plane_angle
            angle_agg /= self.number_of_neighbors
            #print(angle_agg)
            #print(eigen_agg)
            if self.it == self.number_of_neighbors + 1:
                self.initial_eigen_agg = eigen_agg
                self.initial_angle_agg = angle_agg
                if forward:
                    edge_text_mesh: TriangleMesh = o3d.t.geometry.TriangleMesh.create_text(f"Edge {self.seg_id}", depth=4).to_legacy()
                    edge_text_mesh.paint_uniform_color((1, 0, 0))
                    edge_text_location = self.anchor_points[self.seg_id]
                    edge_text_mesh.transform([[0.3, 0, 0, edge_text_location[0]+5], [0, 0.3, 0, edge_text_location[1]+5], [0, 0, 0.3, edge_text_location[2]+3], [0, 0, 0, 1]])
                    vis.add_geometry(edge_text_mesh)
            if eigen_agg > self.eigen_threshold or angle_agg > self.angle_threshold:
                if self.forward:
                    #print(f'broke edge moving forwards with initial angle_agg: {self.initial_angle_agg} and current angle_agg: {angle_agg}')
                    #print('')
                    self.segment_painted_points[self.seg_id] += self.it
                    self.it = 0
                    self.forward = False
                    self.eig_val_dict.clear()
                    self.plane_dir_dict.clear()
                    return
                else:
                    #print(f'broke edge moving backwards with initial eigen_agg: {self.initial_eigen_agg} and current eigen_agg: {eigen_agg}')
                    #print('')
                    self.segment_painted_points[self.seg_id] += self.it
                    self.it = 0
                    self.forward = True
                    self.eig_val_dict.clear()
                    self.plane_dir_dict.clear()
                    self.seg_id += 1
                    return
            else: 
                if self.it > self.number_of_neighbors + 1:
                    np.asarray(self.pcd.colors)[self.sampled_neighborhoods[self.it-1][1:], :] = (0.6, 0.6, 0.6)
                np.asarray(self.pcd.colors)[self.sampled_neighborhoods[self.it][1:], :] = (0, 0, 0)
                scale_factor = 5  # Adjust this factor to scale the length of the eigenvectors appropriately for visualization
                for line in self.line_geometry:
                        vis.remove_geometry(line, reset_bounding_box=False)
                self.line_geometry.clear()
                for i in range(3):  # Assuming 3D, so we have three principal components
                    start_point = mu
                    end_point = mu + np.sqrt(eig_val_norm[i]) * eig_vec[:, i] * scale_factor
                    line_set = o3d.geometry.LineSet(
                        points=o3d.utility.Vector3dVector([start_point, end_point]),
                        lines=o3d.utility.Vector2iVector([[0, 1]]),
                    )
                    line_set.colors = o3d.utility.Vector3dVector([[i == 0, i == 1, i == 2]])  # Color the lines RGB based on principal component order
                    vis.add_geometry(line_set)
                    self.line_geometry.append(line_set)

                self.eigen_agg_text_mesh.clear()
                self.eigen_agg_text_mesh: TriangleMesh = o3d.t.geometry.TriangleMesh.create_text(f'Eigen agg = {eigen_agg:.2f}', depth=2).to_legacy()
                self.eigen_agg_text_mesh.paint_uniform_color((1, 0, 0))
                self.eigen_agg_text_location = sampled_anchor_point + [10,10,3]
                self.eigen_agg_text_mesh.transform([[0.2, 0, 0, self.eigen_agg_text_location[0]], [0, 0.2, 0, self.eigen_agg_text_location[1]], [0, 0, 0.2, self.eigen_agg_text_location[2]], [0, 0, 0, 1]])
                vis.add_geometry(self.eigen_agg_text_mesh)

                self.angle_agg_text_mesh.clear()
                self.angle_agg_text_mesh: TriangleMesh = o3d.t.geometry.TriangleMesh.create_text(f'Angle agg = {angle_agg:.2f}', depth=2).to_legacy()
                self.angle_agg_text_mesh.paint_uniform_color((1, 0, 0))
                self.angle_agg_text_location = sampled_anchor_point + [10,5,3]
                self.angle_agg_text_mesh.transform([[0.2, 0, 0, self.angle_agg_text_location[0]], [0, 0.2, 0, self.angle_agg_text_location[1]], [0, 0, 0.2, self.angle_agg_text_location[2]], [0, 0, 0, 1]])
                vis.add_geometry(self.angle_agg_text_mesh)

                view_ctl = vis.get_view_control()
                lookat = sampled_anchor_point
                zoom = 0.080000000000000002
                front = [-0.024106890455448116,-0.57254772319971181,0.81951690799604338]
                up =  [0.014828165865396817,0.81946017828866602,0.57294427451208185]
                view_ctl.set_lookat(lookat)  # Set the point the camera is looking at
                view_ctl.set_up(up)      # Set the up direction of the camera
                view_ctl.set_front(front)  # Set the front direction of the camera
                view_ctl.set_zoom(zoom)          # Set the zoom factor of the camera
                vis.update_geometry(self.pcd)
            
        self.it += 1

visualization = o3d.visualization.VisualizerWithKeyCallback()
state = CallbackState(anchor_points=anchor_points,pcd_tree=pcd_tree,eigen_threshold=eigen_threshold,number_of_neighbors=number_of_neighbors,sample_dist=sample_dist,
                      radius=radius,intersection_lines=intersection_lines, pcd=pcd, angle_threshold=angle_threshold)  
visualization.create_window()
visualization.get_render_option().background_color = np.asarray([0.95, 0.95, 0.95])
visualization.add_geometry(pcd)
#view_ctl = visualization.get_view_control()
#view_ctl.set_lookat(lookat)  # Set the point the camera is looking at
#view_ctl.set_up(up)      # Set the up direction of the camera
#view_ctl.set_front(front)  # Set the front direction of the camera
#view_ctl.set_zoom(zoom)          # Set the zoom factor of the camera

visualization.register_key_callback(262, state.key_callback)
visualization.run()
visualization.destroy_window()